# Federated Learning Tutorial

Author: Daniyal Shahrokhian

## Problem

Worldline has open sourced some of their data of credit card transactions to
try to predict fraud:

https://www.kaggle.com/mlg-ulb/creditcardfraud

Imagine this dataset is cut in half horizontally. Client_1 has half of the data
and Client_2 has the other half. Neither of them wants to send their raw data to us.
However, we convince them to let our model learn from their data in a federated
setting. Implement a way for our model to train on the combined data of both
Client_1 and Client_2 without either of them sending us any raw data. Compare it with the model with the traditional approach that can see all the data at once.

## Dependencies & Setup

In [27]:

!pip install sklearn
!pip install pandas
!pip install matplotlib
!pip install tensorflow

!pip uninstall --yes tensorboard tb-nightly

!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio
!pip install --quiet --upgrade tensorboard


  Using cached numpy-1.18.5-cp37-cp37m-win_amd64.whl (12.7 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\AbhinavBhattacharjee\\Anaconda3\\Lib\\site-packages\\numpy\\.libs\\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Rolling back uninstall of numpy
  Moving to c:\users\abhinavbhattacharjee\anaconda3\lib\site-packages\numpy-1.19.5.dist-info\
   from c:\users\abhinavbhattacharjee\anaconda3\lib\site-packages\~umpy-1.19.5.dist-info
  Moving to c:\users\abhinavbhattacharjee\anaconda3\lib\site-packages\numpy\.libs\libopenblas.wcdjnk7yvmpzq2me2zzhjjrj3jikndb7.gfortran-win_amd64.dll
   from C:\Users\ABHINA~1\AppData\Local\Temp\pip-uninstall-15f_hbg4\libopenblas.wcdjnk7yvmpzq2me2zzhjjrj3jikndb7.gfortran-win_amd64.dll
  Moving to c:\users\abhinavbhattacharjee\anaconda3\lib\site-packages\numpy\__config__.py
   from C:\Users\ABHINA~1\AppData\Local\Temp\pip-uninstall-z84aju8d\__config__.py
  Moving to c:\users\abhinavbhattacharjee\anaconda3\lib\site-packages\numpy\__init__.py
   from C:\Users\ABHINA~1\AppData\Local\Temp\pip-uninstall-z84aju8d\__init__.py
  Movi

^C


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.3.4 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.19.5 which is incompatible.
tensorflow-federated 0.17.0 requires numpy~=1.18.4, but you have numpy 1.19.5 which is incompatible.


In [29]:
import matplotlib.pyplot as plt


import nest_asyncio
nest_asyncio.apply()

import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall

SEED = 1337
tf.random.set_seed(SEED)

## Data

In [30]:
df = pd.read_csv('creditcard.csv')
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [31]:
# Creating Client_1 and Client_2's splits:
Client_1_df = df[:len(df.index)//2]
Client_2_df = df[len(df.index)//2:]

### Exploratory Analysis

Fraudulent transactions only account for 0.17% of the total transactions. Given the large distribution difference, class weighting applied to the classifier won't cut it, so it is very likely that we will need to rely on under/over-sampling.

In [32]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

When splitting the data horizontally, the class distribution does not change drastically.

In [33]:
Client_1_df['Class'].value_counts()

0    142134
1       269
Name: Class, dtype: int64

In [34]:
Client_2_df['Class'].value_counts()

0    142181
1       223
Name: Class, dtype: int64

As it can be seen by plotting the density estimation of the two datasets, there is some differences between the two. In many Federated scenarios, the data sources are non-i.i.d. (Independent and Identically Distributed). At first glance, this also happens in our dataset, but given the small variance it won't be much of a problem. The only variables that have significant differences are `Time` and `Amount`, the former we will not even include on our classifier.

## Setup

In [36]:
ITRS = 100
BATCH_SIZE = 64

## Federated Learning Approach

### Data Loading

In [37]:
def make_tf_dataset(dataframe, negative_ratio=None, batch_size=None):
    dataset = dataframe.drop(['Time'], axis=1)

    # Class balancing
    pos_df = dataset[dataset['Class'] == 1]
    neg_df = dataset[dataset['Class'] == 0]
    if negative_ratio:
        neg_df = neg_df.iloc[random.sample(range(0, len(neg_df)), len(pos_df)*negative_ratio), :]
    balanced_df = pd.concat([pos_df, neg_df], ignore_index=True, sort=False)

    y = balanced_df.pop('Class')
    
    # Dataset creation
    dataset = tf.data.Dataset.from_tensor_slices((balanced_df.values, y.to_frame().values))
    dataset = dataset.shuffle(2048, seed=SEED)
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [38]:
train_data, val_data = [], []
for client_data in [Client_1_df, Client_2_df]:
    train_df, val_df = train_test_split(client_data, test_size=0.1, random_state=SEED)

    # Scaling
    scaler = MinMaxScaler() 
    train_features = scaler.fit_transform(train_df.drop(['Class'], axis=1))
    val_features = scaler.transform(val_df.drop(['Class'], axis=1))

    train_df[train_df.columns.difference(['Class'])] = train_features
    val_df[val_df.columns.difference(['Class'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df, negative_ratio=10, batch_size=BATCH_SIZE))
    val_data.append(make_tf_dataset(val_df, batch_size=1))

### Model Definition

In [39]:
def input_spec():
    return (
        tf.TensorSpec([None, 29], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def neural_network():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(29,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[BinaryAccuracy(), Precision(), Recall()])

### Training

Each time the `next` method is called, the server model is broadcast to each client using a broadcast function. For each client, one epoch of local training is performed. Each client computes the difference between the client model after training and the initial broadcast model. These model deltas are then aggregated at the server using some aggregation function.

In [40]:
trainer = tff.learning.build_federated_averaging_process(
    neural_network,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam()
)

state = trainer.initialize()
train_hist = []
for i in range(ITRS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{ITRS}", end="")

ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending coro=<ReferenceResolvingExecutor._evaluate() running at C:\Users\AbhinavBhattacharjee\Anaconda3\lib\site-packages\tensorflow_federated\python\core\impl\executors\reference_resolving_executor.py:513> wait_for=<_GatheringFuture pending cb=[Task.__wakeup()]> cb=[Task.__wakeup()]>
ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending coro=<ReferenceResolvingExecutor._evaluate() running at C:\Users\AbhinavBhattacharjee\Anaconda3\lib\site-packages\tensorflow_federated\python\core\impl\executors\reference_resolving_executor.py:513> wait_for=<_GatheringFuture pending cb=[Task.__wakeup()]> cb=[Task.__wakeup()]>
ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending coro=<ReferenceResolvingExecutor._evaluate() running at C:\Users\AbhinavBhattacharjee\Anaconda3\lib\site-packages\tensorflow_federated\python\core\impl\executors\reference_resolving_executor.py:513> wait_for=<_GatheringFuture pe

Run 100/100

### Evaluation

In [41]:
evaluator = tff.learning.build_federated_evaluation(neural_network)

In [42]:
federated_model_results = evaluator(state.model, val_data)
federated_model_results

OrderedDict([('binary_accuracy', 0.99908715),
             ('precision', 0.75),
             ('recall', 0.65217394),
             ('loss', 0.025250781)])

## Single Model with all Data at once (for comparison)

### Data Loading

In [43]:
train_data = train_data[0].concatenate(train_data[1])
val_data = val_data[0].concatenate(val_data[1])

### Model Definition

In [44]:
def neural_network():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(29,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
    
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=[BinaryAccuracy(), Precision(), Recall()],
    )
    
    return model

### Training

In [46]:
model = neural_network()
history = model.fit(train_data, epochs=ITRS)

Epoch 1/100
78/78 [==============================] - 0s 1ms/step - loss: 0.3686 - binary_accuracy: 0.8467 - precision_2: 0.1786 - recall_2: 0.1906
Epoch 2/100
78/78 [==============================] - 0s 1ms/step - loss: 0.1277 - binary_accuracy: 0.9598 - precision_2: 1.0000 - recall_2: 0.5583
Epoch 3/100
78/78 [==============================] - 0s 1ms/step - loss: 0.0802 - binary_accuracy: 0.9796 - precision_2: 0.9971 - recall_2: 0.7780
Epoch 4/100
78/78 [==============================] - ETA: 0s - loss: 0.0722 - binary_accuracy: 0.9830 - precision_2: 0.9967 - recall_2: 0.81 - 0s 2ms/step - loss: 0.0715 - binary_accuracy: 0.9831 - precision_2: 0.9973 - recall_2: 0.8161
Epoch 5/100
78/78 [==============================] - 0s 1ms/step - loss: 0.0682 - binary_accuracy: 0.9835 - precision_2: 0.9919 - recall_2: 0.8251
Epoch 6/100
78/78 [==============================] - 0s 1ms/step - loss: 0.0647 - binary_accuracy: 0.9847 - precision_2: 0.9920 - recall_2: 0.8386
Epoch 7/100
78/78 [=========

### Evaluation

In [47]:
test_scores = model.evaluate(val_data)
traditional_model_results = {
    'loss': test_scores[0],
    'binary_accuracy': test_scores[1],
    'precision': test_scores[2],
    'recall': test_scores[3]
}
traditional_model_results

28482/28482 [==============================] - 25s 875us/step - loss: 0.0431 - binary_accuracy: 0.9920 - precision_2: 0.1479 - recall_2: 0.8261


{'loss': 0.0430823415517807,
 'binary_accuracy': 0.9920300245285034,
 'precision': 0.14785991609096527,
 'recall': 0.8260869383811951}

## Conclusion

Comparing both models:

In [48]:
print(f">>Single model metrics: \t{traditional_model_results}\n")
print(f">>Federated model metrics: \t{dict(federated_model_results)}")

>>Single model metrics: 	{'loss': 0.0430823415517807, 'binary_accuracy': 0.9920300245285034, 'precision': 0.14785991609096527, 'recall': 0.8260869383811951}

>>Federated model metrics: 	{'binary_accuracy': 0.99908715, 'precision': 0.75, 'recall': 0.65217394, 'loss': 0.025250781}


The Federated Learning approach has a better balance between precision and recall, which might be an indicator of better handling of the imbalanced dataset.